In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
X_train = pd.read_csv("../data/train.csv", date_parser="Date", dtype={"StateHoliday" : str})
store = pd.read_csv("../data/store.csv")

In [42]:
print(len(X_train))
X_train.head()

1017209


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [51]:
X_train[(X_train['Sales'] > 0) & (X_train['Open']==0)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday


In [30]:
X_train = X_train[X_train['Open']==1]
print(len(X_train))

844392


In [34]:
X_train.StateHoliday.unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [35]:
store.head()
len(store)

1115